<a href="https://colab.research.google.com/github/ehddnr301/dacon_motion/blob/master/motion_pytorch_hrnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!rm -rf /content/gdrive/MyDrive/dacon-motion/train_imgs_folder

In [3]:
!rm -rf /content/gdrive/MyDrive/dacon-motion/test_imgs_folder

In [4]:
!pip install torchinfo

In [5]:
!pip install albumentations==0.4.6

     |████████████████████████████████| 122kB 7.8MB/s 
     |████████████████████████████████| 952kB 13.2MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-cp37-none-any.whl size=65163 sha256=403dd260a61c838256142dfcfce3d52dcc00e8f81bc9eb9515d355ff0402149d
  Stored in directory: /root/.cache/pip/wheels/c7/f4/89/56d1bee5c421c36c1a951eeb4adcc32fbb82f5344c086efa14
Successfully built albumentations
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [6]:
!git clone https://github.com/ehddnr301/pytorch-image-models.git

Cloning into 'pytorch-image-models'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 4604 (delta 12), reused 26 (delta 8), pack-reused 4572
Receiving objects: 100% (4604/4604), 15.79 MiB | 24.46 MiB/s, done.
Resolving deltas: 100% (3310/3310), done.


In [7]:
!mv pytorch-image-models timmm

In [8]:
!cd timmm && pip install .

Processing /content/timmm
  Created wheel for timm: filename=timm-0.4.4-cp37-none-any.whl size=287653 sha256=32372d31bec671fe653b45ea04ec4ba3b3f6aa06f1db20a6f23a0f32ae5d2216
  Stored in directory: /tmp/pip-ephem-wheel-cache-8i2n9shh/wheels/54/b9/02/8a014ced29a3023ae1cb7c399966644596eecd7265d47a44c7
Successfully built timm


In [9]:
from timmm import timm

In [10]:
!unzip -n '/content/gdrive/MyDrive/dacon-motion/train_imgs.zip' -d '/content/gdrive/MyDrive/dacon-motion/train_imgs_folder'

Archive:  /content/gdrive/MyDrive/dacon-motion/train_imgs.zip
  inflating: /content/gdrive/MyDrive/dacon-motion/train_imgs_folder/001-1-1-01-Z17_A-0000001.jpg  
  inflating: /content/gdrive/MyDrive/dacon-motion/train_imgs_folder/001-1-1-01-Z17_A-0000003.jpg  
  inflating: /content/gdrive/MyDrive/dacon-motion/train_imgs_folder/001-1-1-01-Z17_A-0000005.jpg  
  inflating: /content/gdrive/MyDrive/dacon-motion/train_imgs_folder/001-1-1-01-Z17_A-0000007.jpg  
  inflating: /content/gdrive/MyDrive/dacon-motion/train_imgs_folder/001-1-1-01-Z17_A-0000009.jpg  
  inflating: /content/gdrive/MyDrive/dacon-motion/train_imgs_folder/001-1-1-01-Z17_A-0000011.jpg  
  inflating: /content/gdrive/MyDrive/dacon-motion/train_imgs_folder/001-1-1-01-Z17_A-0000013.jpg  
  inflating: /content/gdrive/MyDrive/dacon-motion/train_imgs_folder/001-1-1-01-Z17_A-0000015.jpg  
  inflating: /content/gdrive/MyDrive/dacon-motion/train_imgs_folder/001-1-1-01-Z17_A-0000017.jpg  
  inflating: /content/gdrive/MyDrive/dacon-moti

In [11]:
!unzip -n '/content/gdrive/MyDrive/dacon-motion/test_imgs.zip' -d '/content/gdrive/MyDrive/dacon-motion/test_imgs_folder'

Archive:  /content/gdrive/MyDrive/dacon-motion/test_imgs.zip
  inflating: /content/gdrive/MyDrive/dacon-motion/test_imgs_folder/649-2-4-32-Z148_A-0000001.jpg  
  inflating: /content/gdrive/MyDrive/dacon-motion/test_imgs_folder/649-2-4-32-Z148_A-0000003.jpg  
  inflating: /content/gdrive/MyDrive/dacon-motion/test_imgs_folder/649-2-4-32-Z148_A-0000005.jpg  
  inflating: /content/gdrive/MyDrive/dacon-motion/test_imgs_folder/649-2-4-32-Z148_A-0000007.jpg  
  inflating: /content/gdrive/MyDrive/dacon-motion/test_imgs_folder/649-2-4-32-Z148_A-0000009.jpg  
  inflating: /content/gdrive/MyDrive/dacon-motion/test_imgs_folder/649-2-4-32-Z148_A-0000011.jpg  
  inflating: /content/gdrive/MyDrive/dacon-motion/test_imgs_folder/649-2-4-32-Z148_A-0000013.jpg  
  inflating: /content/gdrive/MyDrive/dacon-motion/test_imgs_folder/649-2-4-32-Z148_A-0000015.jpg  
  inflating: /content/gdrive/MyDrive/dacon-motion/test_imgs_folder/649-2-4-32-Z148_A-0000017.jpg  
  inflating: /content/gdrive/MyDrive/dacon-motio

In [12]:
import os
from typing import Tuple, Sequence, Callable
import csv
import cv2
import random
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import torch
import torch.optim as optim
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader, random_split
from torchinfo import summary

import albumentations
from albumentations.pytorch import ToTensorV2, ToTensor

from torchvision import transforms
import timm

In [13]:
# random seed
random_seed = 301
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)


IMAGE_HEIGHT = 1024
IMAGE_WIDTH = 1024
IMAGE_CHANNELS=3
EPOCHS=30
BATCH_SIZE = 4

PATH_TRAIN_DATASET='/content/gdrive/MyDrive/dacon-motion/train_imgs_folder/'
PATH_TEST_DATASET='/content/gdrive/MyDrive/dacon-motion/test_imgs_folder/'
PATH_TRAIN_ANS_CSV='/content/gdrive/MyDrive/dacon-motion/train_df.csv'
SUB_DF = '/content/gdrive/MyDrive/dacon-motion/sample_submission.csv'
PATH_TRAIN_CSV='/content/gdrive/MyDrive/dacon-motion/split_train_df.csv'
PATH_VALID_CSV='/content/gdrive/MyDrive/dacon-motion/split_val_df.csv'

In [14]:
df = pd.read_csv(PATH_TRAIN_ANS_CSV)
df.head()

,image,nose_x,nose_y,left_eye_x,left_eye_y,right_eye_x,right_eye_y,left_ear_x,left_ear_y,right_ear_x,right_ear_y,left_shoulder_x,left_shoulder_y,right_shoulder_x,right_shoulder_y,left_elbow_x,left_elbow_y,right_elbow_x,right_elbow_y,left_wrist_x,left_wrist_y,right_wrist_x,right_wrist_y,left_hip_x,left_hip_y,right_hip_x,right_hip_y,left_knee_x,left_knee_y,right_knee_x,right_knee_y,left_ankle_x,left_ankle_y,right_ankle_x,right_ankle_y,neck_x,neck_y,left_palm_x,left_palm_y,right_palm_x,right_palm_y,spine2(back)_x,spine2(back)_y,spine1(waist)_x,spine1(waist)_y,left_instep_x,left_instep_y,right_instep_x,right_instep_y
0,001-1-1-01-Z17_A-0000001.jpg,1046.389631,344.757881,1041.655294,329.820225,1059.429507,334.484230,1020.117796,338.890539,1048.000000,343.000000,992.0,394.0,1054.0,400.0,956.000000,368.000000,1134.0,371.0,1003.497242,327.640085,1078.000000,341.00000,999.0,570.0,1046.0,573.0,995.0,695.0,1054.0,698.0,983.000000,820.000000,1042.0,829.0,1019.107277,373.766222,1013.137360,316.311695,1067.000000,335.000000,1019.484230,455.000000,1026.515770,514.054730,998.578836,826.718013,1063.204067,838.827465
1,001-1-1-01-Z17_A-0000003.jpg,1069.850679,340.711494,1058.608552,324.593690,1075.242111,325.593690,1041.422997,331.694815,1065.593682,333.968459,1010.0,398.0,1077.0,386.0,974.000000,398.000000,1144.0,337.0,1004.000000,346.000000,1094.000000,326.00000,1020.0,557.0,1069.0,567.0,1013.0,583.0,1072.0,697.0,1003.843781,678.797029,1042.0,829.0,1047.279440,362.031898,1017.383112,332.890539,1081.187380,323.000000,1046.953248,454.062706,1058.766231,508.797029,1002.265676,699.062706,1066.376234,841.499445
2,001-1-1-01-Z17_A-0000005.jpg,1084.475902,337.000008,1078.717997,323.757889,1095.648412,325.242119,1061.039884,329.351571,1086.461032,334.109461,1020.0,381.0,1083.0,389.0,984.000000,362.000000,1163.0,372.0,1027.507419,327.383120,1111.000000,340.00000,1016.0,552.0,1061.0,556.0,1018.0,674.0,1070.0,696.0,987.703151,788.867342,1044.0,826.0,1063.029716,358.679953,1042.374777,319.031541,1101.000000,334.000000,1044.538960,442.054730,1052.844144,495.890539,989.437847,808.757889,1066.071417,841.749554
3,001-1-1-01-Z17_A-0000007.jpg,1042.320047,361.452689,1037.907194,344.117804,1050.328382,353.913729,1016.844144,340.913737,1042.164191,359.351579,968.0,392.0,1010.0,411.0,941.812612,339.703151,1109.0,441.0,971.654931,328.648429,1066.812604,383.40631,972.0,571.0,1000.0,559.0,991.0,700.0,1110.0,469.0,991.406302,816.132650,1133.0,591.0,1007.302983,374.179405,981.842327,329.109461,1057.406318,372.461040,982.937294,458.109462,990.375124,507.624866,1001.305177,829.233767,1159.516499,599.389997
4,001-1-1-01-Z17_A-0000009.jpg,1058.046395,343.164191,1046.717997,331.703163,1058.132650,331.781079,1031.258806,338.593690,1049.812620,338.187380,997.0,393.0,1054.0,395.0,961.000000,423.000000,1132.0,360.0,988.676303,357.688297,1080.000000,337.00000,1000.0,567.0,1043.0,570.0,1017.0,673.0,1059.0,701.0,998.406302,784.570501,1040.0,827.0,1036.318945,366.195727,999.777421,349.829291,1069.648429,334.109461,1024.843791,453.687572,1034.391088,510.843791,998.625231,805.218921,1059.625956,839.765102


In [15]:
x_coor = [col for col in df.columns if '_x' in col]
y_coor = [col for col in df.columns if '_y' in col]

In [16]:
df[x_coor] = (df[x_coor] - 426)

In [17]:
df[y_coor] = (df[y_coor] - 56)

In [18]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df.iloc[:,1:])
df.iloc[:,1:] = scaler.transform(df.iloc[:,1:])

# for inverse transformation


In [19]:
df.head(2)

,image,nose_x,nose_y,left_eye_x,left_eye_y,right_eye_x,right_eye_y,left_ear_x,left_ear_y,right_ear_x,right_ear_y,left_shoulder_x,left_shoulder_y,right_shoulder_x,right_shoulder_y,left_elbow_x,left_elbow_y,right_elbow_x,right_elbow_y,left_wrist_x,left_wrist_y,right_wrist_x,right_wrist_y,left_hip_x,left_hip_y,right_hip_x,right_hip_y,left_knee_x,left_knee_y,right_knee_x,right_knee_y,left_ankle_x,left_ankle_y,right_ankle_x,right_ankle_y,neck_x,neck_y,left_palm_x,left_palm_y,right_palm_x,right_palm_y,spine2(back)_x,spine2(back)_y,spine1(waist)_x,spine1(waist)_y,left_instep_x,left_instep_y,right_instep_x,right_instep_y
0,001-1-1-01-Z17_A-0000001.jpg,0.705676,-0.883393,0.643712,-0.903614,0.764675,-0.877807,0.537504,-0.867752,0.737195,-0.838452,0.360363,-0.962454,0.948295,-0.915517,0.031724,-1.659000,1.404571,-1.598692,0.347610,-1.836882,0.955774,-1.800711,0.450565,-0.633531,1.076083,-0.522233,0.166782,0.219659,0.688501,0.267716,-0.014522,0.395551,0.346131,0.468444,0.586447,-0.892244,0.390703,-1.947932,0.832882,-1.819319,0.777074,-0.827095,0.893008,-0.892409,0.049815,0.277597,0.407093,0.370094
1,001-1-1-01-Z17_A-0000003.jpg,0.841782,-0.912497,0.739617,-0.940201,0.853721,-0.940026,0.665269,-0.917665,0.842334,-0.900715,0.496581,-0.926673,1.119318,-1.038022,0.153847,-1.385347,1.471256,-1.898322,0.350765,-1.704232,1.058441,-1.911407,0.707901,-0.853176,1.343288,-0.615817,0.309652,-0.911312,0.825418,0.256973,0.101569,-0.691961,0.346131,0.468444,0.780772,-0.984788,0.416189,-1.831020,0.919722,-1.903592,1.053228,-0.837242,1.286198,-0.972199,0.068407,-0.670380,0.422679,0.389702


In [20]:
# inversed = scaler.inverse_transform(df.iloc[:,1:])
# print(inversed[0])


In [21]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(df, test_size=0.05, random_state=random_seed)

In [22]:
train.head(2)

,image,nose_x,nose_y,left_eye_x,left_eye_y,right_eye_x,right_eye_y,left_ear_x,left_ear_y,right_ear_x,right_ear_y,left_shoulder_x,left_shoulder_y,right_shoulder_x,right_shoulder_y,left_elbow_x,left_elbow_y,right_elbow_x,right_elbow_y,left_wrist_x,left_wrist_y,right_wrist_x,right_wrist_y,left_hip_x,left_hip_y,right_hip_x,right_hip_y,left_knee_x,left_knee_y,right_knee_x,right_knee_y,left_ankle_x,left_ankle_y,right_ankle_x,right_ankle_y,neck_x,neck_y,left_palm_x,left_palm_y,right_palm_x,right_palm_y,spine2(back)_x,spine2(back)_y,spine1(waist)_x,spine1(waist)_y,left_instep_x,left_instep_y,right_instep_x,right_instep_y
1365,201-2-1-10-Z36_D-0000005.jpg,-0.847862,-1.546467,-0.779902,-1.553380,-0.842509,-1.534844,-0.572627,-1.532893,-0.663913,-1.523713,-0.570458,-1.704914,-1.200647,-1.536791,-0.287151,-2.288401,-1.662941,-1.977636,-0.835150,-1.863181,-1.37990,-1.896647,-1.363035,-0.785593,-1.944497,-0.693803,-0.896804,0.785145,-1.228333,0.686714,-0.727427,1.027096,-0.863437,0.860566,-0.800739,-1.505562,-0.954708,-1.844352,-1.263436,-1.786815,-1.314400,-1.478996,-1.685748,-1.378865,-0.775074,1.029723,-0.910743,0.899722
1355,201-2-1-10-Z36_C-0000017.jpg,0.126266,-1.463443,0.181788,-1.483378,0.047239,-1.486454,0.296922,-1.456592,-0.087488,-1.472696,0.522591,-1.414726,-0.383780,-1.427774,1.103689,-2.069479,-1.009427,-1.960011,0.426061,-1.805381,-0.37248,-1.867128,0.266755,-0.599740,-0.562001,-0.553428,0.119159,0.562990,-0.384013,0.611509,-0.053509,0.734429,-0.312652,0.722170,0.082172,-1.434581,0.269826,-1.816144,-0.259959,-1.790427,0.037934,-1.379939,-0.113007,-1.109063,-0.064045,0.769811,-0.296549,0.767631


In [23]:
valid.head(2)

,image,nose_x,nose_y,left_eye_x,left_eye_y,right_eye_x,right_eye_y,left_ear_x,left_ear_y,right_ear_x,right_ear_y,left_shoulder_x,left_shoulder_y,right_shoulder_x,right_shoulder_y,left_elbow_x,left_elbow_y,right_elbow_x,right_elbow_y,left_wrist_x,left_wrist_y,right_wrist_x,right_wrist_y,left_hip_x,left_hip_y,right_hip_x,right_hip_y,left_knee_x,left_knee_y,right_knee_x,right_knee_y,left_ankle_x,left_ankle_y,right_ankle_x,right_ankle_y,neck_x,neck_y,left_palm_x,left_palm_y,right_palm_x,right_palm_y,spine2(back)_x,spine2(back)_y,spine1(waist)_x,spine1(waist)_y,left_instep_x,left_instep_y,right_instep_x,right_instep_y
1321,201-2-1-10-Z36_A-0000013.jpg,1.010886,-0.860073,0.883748,-0.842354,0.993103,-0.849482,0.728698,-0.866993,0.997575,-0.872922,0.466310,-0.846165,1.275469,-0.871765,0.153847,-0.582633,1.63130,-0.735051,0.469984,-0.526561,1.362585,-0.636611,0.585360,-0.329408,1.308435,-0.085509,0.293321,0.631821,0.75696,0.729688,0.189752,0.776642,0.413870,0.793071,0.878763,-0.889726,0.507015,-0.560701,1.237907,-0.590326,1.053698,-0.880590,1.088578,-0.780768,0.233157,0.723328,0.443721,0.751886
3014,514-1-2-24-Z134_B-0000007.jpg,-0.303810,1.066896,-0.460261,1.075350,-0.351406,1.100111,-0.603122,1.298934,-0.475697,1.328559,-0.773851,1.609969,0.064801,2.035675,-0.062431,1.149441,0.84475,1.734820,0.550219,0.434860,1.480887,0.994841,1.399094,0.524360,1.873005,0.847948,1.419883,-1.604750,1.08863,-1.816906,1.629997,-2.546193,1.021192,-3.089761,-0.529478,1.690169,0.617441,0.356915,1.505929,0.935203,0.082219,1.733627,0.908865,1.636464,1.567254,-2.728138,0.905153,-3.304312


In [24]:
train.to_csv(PATH_TRAIN_CSV, index=False)
valid.to_csv(PATH_VALID_CSV, index=False)


In [25]:
class MotionDataset(Dataset):
    def __init__(
        self,
        dir: os.PathLike,
        image_ids: os.PathLike,
        transforms: Sequence[Callable]
    ) -> None:
        self.dir = dir
        self.transforms = transforms

        self.labels = {}
        with open(image_ids, 'r') as f:
            reader = csv.reader(f)
            next(reader)
            for row in reader:
                self.labels[str(row[0])] = list(map(float, row[1:]))

        self.image_ids = list(self.labels.keys())

    def __len__(self) -> float:
        return len(self.image_ids)

    def __getitem__(self, index: float) -> Tuple[Tensor]:
        image_id = self.image_ids[index]
        image = cv2.imread(os.path.join(self.dir, image_id))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        target = np.array(self.labels.get(image_id)).astype(np.float32)

        if self.transforms is not None:
            augmented = self.transforms(image=image)
            image = augmented['image']

        return image, target

In [26]:
transforms_train = albumentations.Compose([
      albumentations.Crop(426, 56, 1450, 1080),
      # albumentations.OneOf([albumentations.HorizontalFlip(p=1),
      #           albumentations.RandomRotate90(p=1),
      #           albumentations.VerticalFlip(p=1)            
      # ], p=0.5),
      albumentations.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
      ToTensorV2()
])

transforms_valid = albumentations.Compose([
    albumentations.Crop(426, 56, 1450, 1080),
    albumentations.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ToTensorV2()
])

transforms_test = albumentations.Compose([
    albumentations.Crop(426, 56, 1450, 1080),
    albumentations.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ToTensorV2()
])


In [27]:
trainset = MotionDataset(PATH_TRAIN_DATASET, PATH_TRAIN_CSV, transforms_train)
valset = MotionDataset(PATH_TRAIN_DATASET, PATH_VALID_CSV, transforms_valid)
testset = MotionDataset(PATH_TEST_DATASET, SUB_DF, transforms_test)

train_loader = DataLoader(trainset, batch_size=2)
val_loader = DataLoader(valset, batch_size=4)
test_loader = DataLoader(testset, batch_size=2)

In [28]:
# trainset[0]

In [29]:
class HRnetModel(nn.Module):
    def __init__(self, num_classes=48, model_name='hrnet_w48', pretrained=True):
        super(HRnetModel, self).__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        self.model.classifier = nn.Linear(2048, num_classes)
        
    def forward(self, x):
        x = self.model(x)

        return x

In [30]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HRnetModel().to(device)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-hrnet/hrnetv2_w48-abd2e6ab.pth" to /root/.cache/torch/hub/checkpoints/hrnetv2_w48-abd2e6ab.pth


In [31]:
print(summary(model, input_size=(1, 3, 1024, 1024), verbose=0))

Layer (type:depth-idx)                        Output Shape              Param #
├─HighResolutionNet: 1-1                      [1, 48]                   --
|    └─Conv2d: 2-1                            [1, 64, 512, 512]         1,728
|    └─BatchNorm2d: 2-2                       [1, 64, 512, 512]         128
|    └─ReLU: 2-3                              [1, 64, 512, 512]         --
|    └─Conv2d: 2-4                            [1, 64, 256, 256]         36,864
|    └─BatchNorm2d: 2-5                       [1, 64, 256, 256]         128
|    └─ReLU: 2-6                              [1, 64, 256, 256]         --
|    └─Sequential: 2-7                        [1, 256, 256, 256]        --
|    |    └─Bottleneck: 3-1                   [1, 256, 256, 256]        75,008
|    |    └─Bottleneck: 3-2                   [1, 256, 256, 256]        70,400
|    |    └─Bottleneck: 3-3                   [1, 256, 256, 256]        70,400
|    |    └─Bottleneck: 3-4                   [1, 256, 256, 256]        70

In [ ]:
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.001)
criterion = nn.MSELoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                  mode='min',
                                                  factor=0.5,
                                                  patience=2,)



for epoch in range(EPOCHS):
    

    for i, (images, targets) in enumerate(train_loader):

        optimizer.zero_grad()

        images = images.to(device)
        targets = targets.to(device)

        model.train()

        with torch.set_grad_enabled(True):
            outputs = model(images)
            loss = criterion(outputs, targets)

            loss.backward()
            optimizer.step()

        if i % 100 == 0:
            outputs = outputs
            
            print(f'{epoch+1}: {loss.item():.5f}')


    with tqdm(val_loader,
            total=val_loader.__len__(),
            unit="batch") as valid_bar:
        for i, (images, targets) in enumerate(valid_bar):
            valid_bar.set_description(f"Valid Epoch {epoch + 1}")
            optimizer.zero_grad()

            images = images.to(device)
            targets = targets.to(device)

            # 모델의 dropoupt, batchnormalization를 eval모드로 설정
            # .forward()에서 중간 노드의 gradient를 계산

            model.eval()

            with torch.no_grad():
                # validation loss만을 계산
                probs  = model(images)
                valid_loss = criterion(probs, targets)

            valid_bar.set_postfix(valid_loss = valid_loss.item())

    scheduler.step(valid_loss)


    if (epoch + 1) % 2 == 0:
        torch.save({
                'epoch': epoch+1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
                }, f'/content/gdrive/MyDrive/dacon-motion/checkpoint/model_hrnet_0307_{epoch + 1}.pt')
    
    print('------------------')

1: 1.43159
1: 0.96889
1: 0.51544
1: 0.37914
1: 0.58511
1: 0.26781
1: 0.36013
1: 0.70704
1: 0.52252
1: 0.26347
1: 0.15261
1: 0.37553
1: 0.30275
1: 0.54762
1: 0.38209
1: 0.19911
1: 0.11649
1: 0.23472
1: 0.08224
1: 0.13373


Valid Epoch 1: 100%|██████████| 53/53 [00:40<00:00,  1.30batch/s, valid_loss=0.493]


------------------
2: 0.22158
2: 0.53564
2: 0.11463
2: 0.14727
2: 0.22551
2: 0.07721
2: 0.11208
2: 0.62770
2: 0.25116
2: 0.12401
2: 0.07193
2: 0.09795
2: 0.10594
2: 0.26605
2: 0.20022


In [ ]:
submit = pd.read_csv(SUB_DF)

In [ ]:
# # 불러와서 예측하기

# PATH = '/content/gdrive/MyDrive/dacon-motion/checkpoint/model_hrnet_0307_20.pt'
# model = HRnetModel().to(device)

# checkpoint = torch.load(PATH)
# model.load_state_dict(checkpoint['model_state_dict'])


In [ ]:
!nvidia-smi

In [ ]:
model.eval()
batch_size = test_loader.batch_size
batch_index = 0
for i, (images, targets) in enumerate(tqdm(test_loader)):
    images = images.to(device)
    targets = targets.to(device)
    outputs = model(images)
    outputs = outputs 
    batch_index = i * batch_size
    submit.iloc[batch_index:batch_index+batch_size, 1:] = \
        outputs.float().squeeze(0).detach().cpu().numpy()

In [ ]:
submit.head(2)

In [ ]:
inversed = scaler.inverse_transform(submit.iloc[:,1:])
print(inversed[0])

In [ ]:
submit.iloc[:,1:] = inversed
submit.head(2)


In [ ]:
submit[x_coor] = (submit[x_coor] + 426)
submit[y_coor] = (submit[y_coor] + 56)

In [ ]:
submit.head(2)

In [ ]:
submit.to_csv('/content/gdrive/MyDrive/dacon-motion/submit_hr_0307.csv', index=False)